In [1]:
import cv2
import mediapipe as mp
import os
import sys
import customtkinter
import tkinter as tk
from tkinter import *
import tkinter.ttk as ttk
from queue import Queue
import threading
import numpy as np
import pandas as pd
import csv
from PIL import Image, ImageTk

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
CSVEXTENTION = "_POINTMARKS.csv"
ROOTPATH  ="./DATABASE"
DATABASEFILE = "POSES_INFORMATION.xlsx"
IMAGEEXTENTION = ('.jpg', '.png', 'jpeg','JPG','.webp')
TXTEXTENTION = "_DESCRIPTION.txt"

In [3]:
def DicttoCSV(inpDict,csvFileName):
    with open(csvFileName, "w") as outfile:
        writerfile = csv.writer(outfile)
        writerfile.writerow(inpDict.keys())
        writerfile.writerows(zip(*inpDict.values()))

In [4]:
def CSVtoDict(fileName):
    outputDict = {}
    df = pd.read_csv(fileName)
    for v in df.columns:
        outputDict[int(v)] = None
        a=[]
        for i in df[v]:
            a.append(int(i))
        outputDict[int(v)] = a
    return outputDict

In [5]:
def loadDescription(fileName):
    with open(fileName) as f:
        lines = f.readlines()
    return lines

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

In [7]:
def ExtractPointsandSaveinCSV(IMAGE_FILES, width = 800, height = 600):
    BG_COLOR = (192, 192, 192)
    TestList = {}
    for keys, b in IMAGE_FILES.items():
        with mp_holistic.Holistic(
            static_image_mode=True,
            model_complexity=2,
            enable_segmentation=True,
            refine_face_landmarks=True) as holistic:
            image = cv2.imread(b)
            down_points = (width, height)
            image = cv2.resize(image, down_points, interpolation= cv2.INTER_LINEAR)
            image_height, image_width, _ = image.shape
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if results.pose_landmarks:
                condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
                bg_image = np.zeros(image.shape, dtype=np.uint8)
                bg_image[:] = BG_COLOR
                annotated_image = image.copy()
                annotated_image = np.where(condition, annotated_image, bg_image)
                
                a = {11 : 11, 12 : 12,
                     13 : 13, 14 : 14,
                     15 : 15, 16 : 16,
                     23 : 23, 24 : 24,
                     25 : 25, 26 : 26,
                     27 : 27, 28 : 28}
                for i in (a.keys()):
                    key = a[i]
                    cordinates = []
                    X = int(np.round(results.pose_landmarks.landmark[key].x * width))
                    Y = int(np.round(results.pose_landmarks.landmark[key].y * height))
                    cordinates.append(X)
                    cordinates.append(Y)
                    a[i] = cordinates
                
                oldKeyName = b
                num = oldKeyName.rfind('.')
                newKeyName = oldKeyName[0:num]
                print(newKeyName)
                DicttoCSV(a, (newKeyName+CSVEXTENTION))
                
                
               # print(newKeyName," ",a,"\n")
                
#                 TestList[b]=a
            
#         oldKeyName = b
#         num = oldKeyName.rfind('.')
#         newKeyName = oldKeyName[0:num]
#         TestList[newKeyName] = TestList.pop(oldKeyName)
#     print(TestList)

In [8]:
def restore_landmarks(pose_map,image,color,thickness,color_circle = (255,0,0), radious = 10, text = ' '):
    
    for indx, pose in enumerate(pose_map.keys()):
        cv2.circle(image, pose_map[pose], radious, color_circle, -thickness)
        if pose == 11:
            cv2.line(image, pose_map[pose], pose_map[13], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[23], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[12], color, thickness)
        elif pose == 12:
            cv2.line(image, pose_map[pose], pose_map[14], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[24], color, thickness)
        elif pose == 13:
            cv2.line(image, pose_map[pose], pose_map[15], color, thickness)
        elif pose == 14:
            cv2.line(image, pose_map[pose], pose_map[16], color, thickness)
        elif pose == 23:
            cv2.line(image, pose_map[pose], pose_map[25], color, thickness)
            cv2.line(image, pose_map[pose], pose_map[24], color, thickness)
        elif pose == 24:
            cv2.line(image, pose_map[pose], pose_map[26], color, thickness)
        elif pose == 25:
            cv2.line(image, pose_map[pose], pose_map[27], color, thickness)
        elif pose == 26:
            cv2.line(image, pose_map[pose], pose_map[28], color, thickness)
        position = (10,50)
        cv2.putText(image,text,position, cv2.FONT_HERSHEY_SIMPLEX, 2,(209, 80, 0, 255),2)

In [9]:
def calculate_accurancy(standart_pose, user_pose):
    accurancy = {}
    standart_list = list(standart_pose.items())
    user_list = list(user_pose.items())
    for i in range(len(standart_pose)):
        point=standart_list[i][0]
        standart = standart_list[i][1]
        value = user_list[i][1]
        absolute = [abs(standart[0]-value[0]),abs(standart[1]-value[1])]
        try:
            a = absolute[0]/value[0]
            b = absolute[1]/value[1]
        except ZeroDivisionError:
            a = value[0]
            b = value[1]
        relative = [round((a*100),1),round((b*100),1)]
        accurancy[point]=relative
    return accurancy

In [10]:
def save_landmarks(results, width, height):
    a= results.pose_landmarks
    landmark_map = {11 : 11, 12 : 12,
                    13 : 13, 14 : 14,
                    15 : 15, 16 : 16,
                    23 : 23, 24 : 24,
                    25 : 25, 26 : 26,
                    27 : 27, 28 : 28}
    

    for i in (landmark_map.keys()):
        key = landmark_map[i]
        cordinates = []
        X = int(np.round(results.pose_landmarks.landmark[key].x * width))
        Y = int(np.round(results.pose_landmarks.landmark[key].y * height))
        cordinates.append(X)
        cordinates.append(Y)
        landmark_map[i] = cordinates
    return landmark_map

In [11]:
def initDataBase():
    DATA_STRUCT = {}
    dataframe = pd.read_excel((os.path.join(ROOTPATH,DATABASEFILE)))
    a = dataframe.columns.tolist()
    for i in a:
        DATA_STRUCT[i]=dataframe[i].tolist()
    return DATA_STRUCT

In [12]:
def filePathExtract(lvl, infoDict, fileFormat):
    ext = None
    outputDict = {}
    if fileFormat == 'txt':
        ext = TXTEXTENTION
    elif fileFormat == 'img':
        ext = IMAGEEXTENTION
    elif fileFormat == 'csv':
        ext = CSVEXTENTION
        
    for i in infoDict:
        filePath=os.path.join(ROOTPATH, lvl, i)
        files = os.listdir(filePath)
        for file in files:
            if file.endswith(ext):
                outputDict[i] = os.path.join(filePath, file)
    return outputDict



In [13]:
# lvl = "BEGINER"
# a = initDataBase(lvl)
# showImg = filePathExtract(lvl, POSES,'img')
# showTxt = filePathExtract(lvl, POSES,'txt')
# showCsv = filePathExtract(lvl, POSES,'csv')

In [ ]:
lvl  = initDataBase()
lvlB = lvl['BEGINNER']
lvlB
showTxt = filePathExtract('BEGINNER', lvlB,'txt')
a = loadDescription(showTxt['CHILD'])
a